### Data is cleaned and embedded using MPNet model. 

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import nltk
import re
from nltk.corpus import stopwords#, PlainTextCorpusReader
from nltk import word_tokenize, ngrams
from nltk.stem import WordNetLemmatizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from transformers import LlamaTokenizer
from sentence_transformers import SentenceTransformer
from datetime import datetime, date, timedelta
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, balanced_accuracy_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler

import warnings
warnings.filterwarnings('ignore')

## Data Partitioning

In [3]:
import os
os.getcwd()

'C:\\Users\\mehdi.sadeghi\\OneDrive - Georgia Institute of Technology\\Gatech\\ISYE 6740\\Project\\archive'

In [4]:
new_directory = 'C:\\Users\\mehdi.sadeghi\\OneDrive - Georgia Institute of Technology\\Gatech\\ISYE 6740\\Project\\Emotions-main\\Emotions-main\\02_Model'

# Change the current working directory
os.chdir(new_directory)

# Verify the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: C:\Users\mehdi.sadeghi\OneDrive - Georgia Institute of Technology\Gatech\ISYE 6740\Project\Emotions-main\Emotions-main\02_Model


In [94]:
import pandas as pd

In [101]:
#embd_data=pd.read_csv('./embeddings/mpnet_embed_df_part400000.csv') 
#clean_data=pd.read_csv('./clean_text.csv') 

,Unnamed: 0.1,Unnamed: 0,text,label,clean_text
416788,416804,416804,i feel like telling these horny devils to find...,2,feel like tell horny devil find site suit sort...
416789,416805,416805,i began to realize that when i was feeling agi...,3,begin realize feel agitate restless think go dish
416790,416806,416806,i feel very curious be why previous early dawn...,5,feel curious previous early dawn time seek tro...
416791,416807,416807,i feel that becuase of the tyranical nature of...,3,feel because tyrannical nature government el s...
416792,416808,416808,i think that after i had spent some time inves...,5,think spend time investigate surround things s...


In [ ]:
embd_data.shape

In [ ]:
X = embd_data.drop('label', axis=1)
y = embd_data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [12]:
# Saving Train and test data for later use in all modeling efforts
train.to_csv('./train_df.csv')
test.to_csv('./test_df.csv')

In [11]:
train.head()

,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,...,c_761,c_762,c_763,c_764,c_765,c_766,c_767,text_WO_stopwords,text,label
342462,0.005049,-0.004521,-0.033797,-0.011138,-0.024820,0.018492,-0.015220,0.012329,-0.017906,0.047792,...,0.021197,0.024708,-0.039280,-0.020336,0.022013,-0.060829,-0.017347,feel im torture slowly today,i feel as if im being tortured very slowly today,4
87990,-0.039426,-0.004666,-0.022700,0.006759,-0.026521,0.036558,-0.009668,-0.055998,0.037705,-0.007174,...,-0.043236,0.020760,0.006370,0.007713,-0.046003,-0.000759,-0.012333,ill feel really bad,ill feel really bad,0
381594,0.027580,0.060547,-0.028398,-0.019655,0.017900,-0.028050,-0.067037,0.006156,-0.047099,0.023724,...,0.000482,0.009941,-0.061392,-0.028997,0.022805,0.026034,-0.006839,feel content whatever news tell know rise chal...,i feel content with whatever news we are told ...,1
208217,-0.002705,0.047667,-0.001580,-0.026343,0.015890,0.024639,-0.091406,0.014512,0.029813,0.026739,...,0.031983,-0.009349,-0.035115,0.008799,0.015763,0.038264,-0.037314,love feel invigorate arrive work,i love feeling invigorated when i arrive at work,1
376976,0.014338,-0.047544,0.025093,0.011402,-0.006114,-0.007457,-0.060139,-0.041030,-0.072453,0.019001,...,-0.019103,0.007613,-0.013725,-0.008387,0.000549,0.007308,0.002215,feel acceptable use first many polish get rece...,i feel is acceptable i used the first of many ...,1


In [1]:
!pip install imblearn
!pip install nltk
!pip install transformers
!pip install Sentence_Transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# 1- Using Embedded data from MPNet model and LLama Lite pre-trained model as classifier for emotion classification

In [5]:
class EmbeddedDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels
    def __len__(self):
        return len(self.embeddings)
    def __getitem__(self, idx):
        embedding=self.embeddings[idx]
        embedding=torch.tensor(embedding, dtype=torch.float32)
        embedding=embedding.unsqueeze(0)
        return embedding, self.labels[idx]

In [13]:
import os
current_directory = os.getcwd()
print("Current directory:", current_directory)

Current directory: /mnt/code/notebooks


In [13]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [15]:
print('train: ', len(train), '\ntest: ', len(test)) #, '\nval: ', len(val)

train:  312606 
test:  104203


In [16]:
train['label'] = train['label'].astype('category')
test['label'] = test['label'].astype('category')

In [18]:
train.dtypes

c_0                   float64
c_1                   float64
c_2                   float64
c_3                   float64
c_4                   float64
                       ...   
c_766                 float64
c_767                 float64
text_WO_stopwords      object
text                   object
label                category
Length: 771, dtype: object

## Prepare model input for model training and validation

In [36]:
train_embeddings= np.stack(train.iloc[:,:768].values)
train_labels= train['label'].values
train_dataset= EmbeddedDataset(train_embeddings, train_labels)
train_loader= DataLoader(train_dataset, batch_size=128, shuffle=True)

In [38]:
test_embeddings= np.stack(test.iloc[:,:768].values)
test_labels= test['label'].values
test_dataset= EmbeddedDataset(test_embeddings, test_labels)
test_loader= DataLoader(test_dataset, batch_size=128, shuffle=True)

In [26]:
num_labels= len(train['label'].unique())
print(num_labels)

6


In [39]:
from transformers import LlamaForSequenceClassification, AdamW
model_name='skeskinen/llama-lite-134m'
model=LlamaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
optimizer= AdamW(model.parameters(), lr=2e-5, weight_decay=1e-2)
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)
model.to(device)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at skeskinen/llama-lite-134m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device:  cuda


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 768, padding_idx=0)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (o_proj): Linear(in_features=768, out_features=768, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=768, out_features=2048, bias=False)
          (up_proj): Linear(in_features=768, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=768, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
      (1): LlamaDecoderLayer(
        (self_att

In [28]:
num_epochs=30
best_test_loss=float('inf')
patience=3
patience_counter=0
best_accuracy=-1
for epoch in range(num_epochs):
    model.train()
    total_loss= 0
    for batch in train_loader:
        embeddings=batch[0].to(device)
        labels=batch[1].to(device)
        optimizer.zero_grad()
        outputs= model(inputs_embeds=embeddings, labels=labels)
        loss= outputs.loss
        total_loss+= loss.item()
        
        loss.backward()
        optimizer.step()
    average_loss= total_loss / len(train_loader)
    print (f'Epoch: {epoch+1}, Loss: {average_loss}')
    
    model.eval()
    test_loss= 0
    correct_predictions= 0
    with torch.no_grad():
        for batch in test_loader:
            embeddings=batch[0].to(device)
            labels=batch[1].to(device)
            
            outputs= model(inputs_embeds=embeddings, labels=labels)
            test_loss+= outputs.loss.item()
            
            logits= outputs.logits
            _, predicted_labels= torch.max(logits, dim=1)
            correct_predictions+= torch.sum(predicted_labels==labels)
    average_test_loss = test_loss/len(test_loader)
    accuracy=correct_predictions/len(test_dataset)
    print(f'Test Loss: {average_test_loss}, Accuracy: {accuracy}')
    
    if accuracy > best_accuracy:
        best_accuracy= accuracy
        patience_counter=0
        torch.save({'model.state_dict()': model.state_dict(), 'optimizer.state_dict()': optimizer.state_dict()}, 'model.pt')
        patience_counter=0
    else:
        patience_counter+=1
    if patience_counter>=patience:
        print('Early stopping Triggered')
        break

Epoch: 1, Loss: 0.47116596780176606
Test Loss: 0.6818288690473404, Accuracy: 0.750822901725769
Epoch: 2, Loss: 0.25431708495673394
Test Loss: 0.7496792136891488, Accuracy: 0.7564273476600647
Epoch: 3, Loss: 0.1716895796681544
Test Loss: 0.8472883035251699, Accuracy: 0.7544216513633728
Epoch: 4, Loss: 0.13620078459216822
Test Loss: 0.9292120997525432, Accuracy: 0.7511683702468872
Epoch: 5, Loss: 0.12082830793036509
Test Loss: 0.9896049231839327, Accuracy: 0.7497192621231079
Early stopping Triggered


# 2- Balancing train dataset for potential model improvement and use same LLama pre-trained model for fine-tuning and compare the results

In [19]:
class_counts = train['label'].value_counts()
# Sort the counts from smallest to largest
class_counts_sorted = class_counts.sort_values()
class_counts_sorted

5     11229
2     25915
4     35784
3     42988
0     90890
1    105800
Name: label, dtype: int64

In [20]:
X_train=train.iloc[:,:768]
y_train=train['label']
sm=SMOTE(random_state=25, n_jobs=-1, k_neighbors=5)
X_train_sm, y_train_sm= sm.fit_resample(X_train, y_train)

In [21]:
X_train_sm.shape

(634800, 768)

In [22]:
y_train_sm.value_counts()

0    105800
1    105800
2    105800
3    105800
4    105800
5    105800
Name: label, dtype: int64

In [23]:
train_embeddings= np.stack(X_train_sm.iloc[:,:768].values)
train_labels= y_train_sm.values
train_dataset= EmbeddedDataset(train_embeddings, train_labels)
train_loader= DataLoader(train_dataset, batch_size=128, shuffle=True)

In [24]:
test_embeddings= np.stack(test.iloc[:,:768].values)
test_labels= test['label'].values
test_dataset= EmbeddedDataset(test_embeddings, test_labels)
test_loader= DataLoader(test_dataset, batch_size=128, shuffle=True)

In [37]:
num_labels= len(train['label'].unique())
print(num_labels)

6


In [27]:
from transformers import LlamaForSequenceClassification, AdamW
model_name='skeskinen/llama-lite-134m'
model=LlamaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
optimizer= AdamW(model.parameters(), lr=2e-5, weight_decay=1e-2)
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)
model.to(device)

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at skeskinen/llama-lite-134m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device:  cuda


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 768, padding_idx=0)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (o_proj): Linear(in_features=768, out_features=768, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=768, out_features=2048, bias=False)
          (up_proj): Linear(in_features=768, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=768, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
      (1): LlamaDecoderLayer(
        (self_att

In [35]:
train.shape , test.shape

((312606, 771), (104203, 771))

In [40]:
num_epochs=30
best_test_loss=float('inf')
patience=3
patience_counter=0
best_accuracy=-1
for epoch in range(num_epochs):
    model.train()
    total_loss= 0
    for batch in train_loader:
        embeddings=batch[0].to(device)
        labels=batch[1].to(device)
        optimizer.zero_grad()
        outputs= model(inputs_embeds=embeddings, labels=labels)
        loss= outputs.loss
        total_loss+= loss.item()
        
        loss.backward()
        optimizer.step()
    average_loss= total_loss / len(train_loader)
    print (f'Epoch: {epoch+1}, Loss: {average_loss}')
    
    model.eval()
    test_loss= 0
    correct_predictions= 0
    with torch.no_grad():
        for batch in test_loader:
            embeddings=batch[0].to(device)
            labels=batch[1].to(device)
            
            outputs= model(inputs_embeds=embeddings, labels=labels)
            test_loss+= outputs.loss.item()
            
            logits= outputs.logits
            _, predicted_labels= torch.max(logits, dim=1)
            correct_predictions+= torch.sum(predicted_labels==labels)
    average_test_loss = test_loss/len(test_loader)
    accuracy=correct_predictions/len(test_dataset)
    print(f'Test Loss: {average_test_loss}, Accuracy: {accuracy}')
    
    if accuracy > best_accuracy:
        best_accuracy= accuracy
        patience_counter=0
        torch.save({'model.state_dict()': model.state_dict(), 'optimizer.state_dict()': optimizer.state_dict()}, 'model_unbalanced.pt')
        patience_counter=0
    else:
        patience_counter+=1
    if patience_counter>=patience:
        print('Early stopping Triggered')
        break

Epoch: 1, Loss: 0.6910204766863348
Test Loss: 0.6035168456884981, Accuracy: 0.767338752746582
Epoch: 2, Loss: 0.4778885061117259
Test Loss: 0.5801429836662269, Accuracy: 0.7747953534126282
Epoch: 3, Loss: 0.31834184839205326
Test Loss: 0.6402754418323376, Accuracy: 0.7651699185371399
Epoch: 4, Loss: 0.20605498998503288
Test Loss: 0.7783686681759138, Accuracy: 0.7620317935943604
Epoch: 5, Loss: 0.15999099656915108
Test Loss: 0.8979340281954572, Accuracy: 0.7577421069145203
Early stopping Triggered


### Balancing data slightly improved the performance of the model, which cause increase in accuracy from 75.6% on epoch 2 to 77.5% again on epoch 2 when using balanced data.